TO DO:
- Consider geospatial uses of data
- Value counts on bird species to get top 10
- Create subset of birds in top 10 & group by year
- Compare USFWS birds df top 10 with outside USFWS birds df top 10

IF YOU HAVE TIME:
- try to webscrape polygons off of USFWS website & group by wildlife region

COMPLETED:
- Write a read me
- Look into mixed data type columns in birding data - will read in as str due to "X" in some rows
- Look into “sensitive species” list
- Figure out how to read in specific tabs in the xlsx files for FIA
- Separate observation date into year columm to group by 
- Clean tables to drop extraneous columns
- Figure out what years to restrict data
- compile list of birds into one df
- Merge birds table with USFWS table to get specific region & download doc off data.gov website
- Separate birds tables into USFWS regions & not
- create new column for seasons of birding

USEFUL CSV TO EXPORT:
- usfws_top10
- usfws_next_top10
- birds_by_season

General notes:
- Years to look at: 2007-2016
- 1st choice: group by wildlife region; else group by county
- 

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# reading in birds, 2007 to 2016
birds = pd.read_csv('../data/eBird_2007_to_2016_TN/eBird_2007_to_2016_TN.txt', sep='\t')

# cleaning birds df
birds = birds[['GLOBAL UNIQUE IDENTIFIER', 'OBSERVATION DATE', 'TAXONOMIC ORDER', 'CATEGORY', 'COMMON NAME', 'SCIENTIFIC NAME', 'AGE/SEX', 'COUNTRY', 'STATE', 'COUNTY', 'IBA CODE', 'BCR CODE', 'USFWS CODE', 'LATITUDE', 'LONGITUDE', 'OBSERVER ID', 'TRIP COMMENTS']]
birds.columns = ['global_unique_identifier', 'observation_date', 'tax_order', 'category', 'common_name', 'scientific_name', 'age_sex', 'country', 'state', 'county', 'iba_code', 'bcr_code', 'usfws_code', 'lat', 'long', 'observer_id', 'trip_comments']

# creating additional column for observation year
birds['observation_year'] = [x[:4] for x in birds.observation_date]
birds = birds.astype({'observation_year':int})
birds.head()

C:\Users\danil\AppData\Local\Temp\ipykernel_9880\3231252845.py:2: DtypeWarning: Columns (11,12,13,23,46) have mixed types. Specify dtype option on import or set low_memory=False.
  birds = pd.read_csv('../data/eBird_2007_to_2016_TN/eBird_2007_to_2016_TN.txt', sep='\t')


,global_unique_identifier,observation_date,tax_order,category,common_name,scientific_name,age_sex,country,state,county,iba_code,bcr_code,usfws_code,lat,long,observer_id,trip_comments,observation_year
0,URN:CornellLabOfOrnithology:EBIRD:OBS36173336,2007-01-18,21054,species,American Crow,Corvus brachyrhynchos,NaN,United States,Tennessee,Anderson,NaN,28,NaN,36.116386,-84.110001,obsr104960,NaN,2007
1,URN:CornellLabOfOrnithology:EBIRD:OBS269070598,2007-01-16,21054,species,American Crow,Corvus brachyrhynchos,NaN,United States,Tennessee,Anderson,NaN,28,NaN,36.116386,-84.110001,obsr242764,NaN,2007
2,URN:CornellLabOfOrnithology:EBIRD:OBS36173326,2007-01-18,11697,species,American Kestrel,Falco sparverius,NaN,United States,Tennessee,Anderson,NaN,28,NaN,36.116386,-84.110001,obsr104960,NaN,2007
3,URN:CornellLabOfOrnithology:EBIRD:OBS36173330,2007-01-18,689,species,Bufflehead,Bucephala albeola,NaN,United States,Tennessee,Anderson,NaN,28,NaN,36.116386,-84.110001,obsr104960,NaN,2007
4,URN:CornellLabOfOrnithology:EBIRD:OBS269070604,2007-01-16,689,species,Bufflehead,Bucephala albeola,NaN,United States,Tennessee,Anderson,NaN,28,NaN,36.116386,-84.110001,obsr242764,NaN,2007


In [3]:
# reading in sensitive species list
sensitive_species = pd.read_csv('../data/sensitive_species_2000_2020_TN.txt', sep='\t')
sensitive_species = sensitive_species[['GLOBAL UNIQUE IDENTIFIER', 'OBSERVATION DATE', 'TAXONOMIC ORDER', 'CATEGORY', 'COMMON NAME', 'SCIENTIFIC NAME', 'AGE/SEX', 'COUNTRY', 'STATE', 'COUNTY', 'IBA CODE', 'BCR CODE', 'USFWS CODE', 'LATITUDE', 'LONGITUDE', 'OBSERVER ID', 'TRIP COMMENTS']]
sensitive_species.columns = ['global_unique_identifier', 'observation_date', 'tax_order', 'category', 'common_name', 'scientific_name', 'age_sex', 'country', 'state', 'county', 'iba_code', 'bcr_code', 'usfws_code', 'lat', 'long', 'observer_id', 'trip_comments']

# creating additional column for observation year
sensitive_species['observation_year'] = [x[:4] for x in sensitive_species.observation_date]
sensitive_species = sensitive_species.astype({'observation_year':int})

# restricting to relevant years
sensitive_species = sensitive_species.loc[(sensitive_species.observation_year < 2017)&(sensitive_species.observation_year > 2006)].sort_values('observation_year').reset_index(drop=True)

In [4]:
# importing tn counties shape
tn_counties = gpd.read_file('../data/county/tncounty.shp')

In [5]:
# reading in USFWS codes
usfws_codes = pd.read_csv('../data/eBird_2007_to_2016_TN/USFWSCodes.txt', sep='\t')

# cleaning usfws codes df
usfws_codes = usfws_codes.reset_index()
usfws_codes['usfws_name'] = np.where(usfws_codes['USFWS NAME'].isnull(), usfws_codes['USFWS CODE'], usfws_codes['USFWS NAME'])
usfws_codes = usfws_codes[['index','usfws_name']]
usfws_codes.columns = ['usfws_code', 'usfws_name']

### Adding columns for grouping

In [6]:
# creating seasons column: spring migration, fall migration, and offseason

# creating month column
birds['observation_month'] = [x[5:7] for x in birds.observation_date]
birds = birds.astype({'observation_month':int})

# spring: March to May
# fall: Sept to Oct

# # categorizing season based on date
for index, row in birds.iterrows():
    if (row['observation_month'] <= 5)&(row['observation_month'] >= 3):
        birds.loc[index, 'season'] = 'spring migration'
    elif (row['observation_month'] <= 10)&(row['observation_month'] >= 9):
        birds.loc[index, 'season'] = 'fall migration'
    else:
        birds.loc[index, 'season'] = 'offseason'

In [107]:
birds_by_season = birds.groupby('season').count()[['global_unique_identifier']]
birds_by_season.columns = ['count']
birds_by_season = birds_by_season.reset_index()
birds_by_season

,season,count
0,fall migration,419934
1,offseason,1302398
2,spring migration,1091063


In [14]:
# subset of fall & spring migration
fall_birds = birds.loc[birds.season == 'fall migration']
spring_birds = birds.loc[birds.season == 'spring migration']

In [15]:
# creating subset of birds sighted within usfws polygon:
birds_usfws = birds.loc[~birds.usfws_code.isna()]

# joining birds_usfws with usfws codes
birds_usfws = pd.merge(birds_usfws, usfws_codes, on='usfws_code', how='left')

birds_usfws.head()

,global_unique_identifier,observation_date,tax_order,category,common_name,scientific_name,age_sex,country,state,county,...,bcr_code,usfws_code,lat,long,observer_id,trip_comments,observation_year,observation_month,season,usfws_name
0,URN:CornellLabOfOrnithology:EBIRD:OBS36528641,2007-01-29,527,species,American Black Duck,Anas rubripes,NaN,United States,Tennessee,Decatur,...,24,USFWS_723,35.688577,-88.031288,obsr56053,"Mostly fair, wind L-M, -2 to -1C",2007,1,offseason,TENNESSEE NATIONAL WILDLIFE REFUGE
1,URN:CornellLabOfOrnithology:EBIRD:OBS36528636,2007-01-29,21054,species,American Crow,Corvus brachyrhynchos,NaN,United States,Tennessee,Decatur,...,24,USFWS_723,35.688577,-88.031288,obsr56053,"Mostly fair, wind L-M, -2 to -1C",2007,1,offseason,TENNESSEE NATIONAL WILDLIFE REFUGE
2,URN:CornellLabOfOrnithology:EBIRD:OBS36528647,2007-01-29,27969,species,American Robin,Turdus migratorius,NaN,United States,Tennessee,Decatur,...,24,USFWS_723,35.688577,-88.031288,obsr56053,"Mostly fair, wind L-M, -2 to -1C",2007,1,offseason,TENNESSEE NATIONAL WILDLIFE REFUGE
3,URN:CornellLabOfOrnithology:EBIRD:OBS36528637,2007-01-29,477,species,American Wigeon,Mareca americana,NaN,United States,Tennessee,Decatur,...,24,USFWS_723,35.688577,-88.031288,obsr56053,"Mostly fair, wind L-M, -2 to -1C",2007,1,offseason,TENNESSEE NATIONAL WILDLIFE REFUGE
4,URN:CornellLabOfOrnithology:EBIRD:OBS36528629,2007-01-29,20786,species,Blue Jay,Cyanocitta cristata,NaN,United States,Tennessee,Decatur,...,24,USFWS_723,35.688577,-88.031288,obsr56053,"Mostly fair, wind L-M, -2 to -1C",2007,1,offseason,TENNESSEE NATIONAL WILDLIFE REFUGE


In [16]:
# creating separate list of sightings outside of USFWS regions
birds_outside_usfws = birds.loc[birds.usfws_code.isna()]
birds_outside_usfws.drop(columns=['usfws_code', 'iba_code'])
birds_outside_usfws['within_park'] = False

birds_outside_usfws.head()

C:\Users\danil\AppData\Local\Temp\ipykernel_9880\1810162964.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  birds_outside_usfws['within_park'] = False


,global_unique_identifier,observation_date,tax_order,category,common_name,scientific_name,age_sex,country,state,county,...,bcr_code,usfws_code,lat,long,observer_id,trip_comments,observation_year,observation_month,season,within_park
0,URN:CornellLabOfOrnithology:EBIRD:OBS36173336,2007-01-18,21054,species,American Crow,Corvus brachyrhynchos,NaN,United States,Tennessee,Anderson,...,28,NaN,36.116386,-84.110001,obsr104960,NaN,2007,1,offseason,False
1,URN:CornellLabOfOrnithology:EBIRD:OBS269070598,2007-01-16,21054,species,American Crow,Corvus brachyrhynchos,NaN,United States,Tennessee,Anderson,...,28,NaN,36.116386,-84.110001,obsr242764,NaN,2007,1,offseason,False
2,URN:CornellLabOfOrnithology:EBIRD:OBS36173326,2007-01-18,11697,species,American Kestrel,Falco sparverius,NaN,United States,Tennessee,Anderson,...,28,NaN,36.116386,-84.110001,obsr104960,NaN,2007,1,offseason,False
3,URN:CornellLabOfOrnithology:EBIRD:OBS36173330,2007-01-18,689,species,Bufflehead,Bucephala albeola,NaN,United States,Tennessee,Anderson,...,28,NaN,36.116386,-84.110001,obsr104960,NaN,2007,1,offseason,False
4,URN:CornellLabOfOrnithology:EBIRD:OBS269070604,2007-01-16,689,species,Bufflehead,Bucephala albeola,NaN,United States,Tennessee,Anderson,...,28,NaN,36.116386,-84.110001,obsr242764,NaN,2007,1,offseason,False


## EDA

In [17]:
birds_outside_usfws.common_name.value_counts().head(10)

common_name
Northern Cardinal         98383
American Crow             85412
Carolina Chickadee        81991
Blue Jay                  79395
Carolina Wren             77783
Tufted Titmouse           74262
Mourning Dove             73080
American Robin            70680
Red-bellied Woodpecker    63031
Northern Mockingbird      59480
Name: count, dtype: int64

In [51]:
# creating subset called outside usfws top 10, grouping by year & common name
outside_usfws_top10 = birds_outside_usfws.loc[birds_outside_usfws.common_name.isin(['Northern Cardinal', 'American Crow', 'Carolina Chickadee', 'Blue Jay', 'Carolina Wren', 'Tufted Titmouse', 'Mourning Dove', 'American Robin', 'Red-bellied Woodpecker', 'Northern Mockingbird'])]
outside_usfws_top10 = outside_usfws_top10.groupby(['observation_year', 'common_name']).count()[['global_unique_identifier']].reset_index()
outside_usfws_top10.columns = ['observation_year', 'common_name', 'count']

# adding total count per year
total_birds = outside_usfws_top10.groupby('observation_year').sum()[['count']].reset_index()
total_birds['common_name'] = 'all birds'
outside_usfws_top10 = pd.concat([outside_usfws_top10, total_birds]).sort_values('observation_year')

outside_usfws_top10
# .loc[outside_usfws_top10.common_name == 'Northern Mockingbird']
# df.loc[df['channel'].isin(['sale','fullprice'])]

,observation_year,common_name,count
0,2007,American Crow,1960
1,2007,American Robin,1611
2,2007,Blue Jay,2185
3,2007,Carolina Chickadee,2228
4,2007,Carolina Wren,2183
...,...,...,...
93,2016,Carolina Chickadee,17894
92,2016,Blue Jay,17560
91,2016,American Robin,16105
98,2016,Red-bellied Woodpecker,14534


In [93]:
birds_usfws.common_name.value_counts().head(20)

common_name
Great Blue Heron            2399
American Crow               2171
Northern Cardinal           2002
Killdeer                    1945
Canada Goose                1807
Carolina Wren               1753
Mallard                     1750
Red-bellied Woodpecker      1603
Bald Eagle                  1579
Tufted Titmouse             1572
Turkey Vulture              1523
Double-crested Cormorant    1506
Blue Jay                    1469
Ring-billed Gull            1453
Carolina Chickadee          1451
Red-winged Blackbird        1415
Eastern Bluebird            1334
Pied-billed Grebe           1317
Great Egret                 1266
Mourning Dove               1252
Name: count, dtype: int64

In [95]:
# creating subset called usfws top 10, grouping by year & common name
usfws_top10 = birds_usfws.loc[birds_usfws.common_name.isin(['Great Blue Heron', 'American Crow', 'Northern Cardinal', 'Killdeer', 'Canada Goose', 'Carolina Wren', 'Mallard', 'Red-bellied Woodpecker', 'Bald Eagle', 'Tufted Titmouse'])]
usfws_top10 = usfws_top10.groupby(['observation_year', 'common_name']).count()[['global_unique_identifier']].reset_index()
usfws_top10.columns = ['observation_year', 'common_name', 'count']

# adding total count per year as a column
total_birds = usfws_top10.groupby('observation_year').sum()[['count']]
total_birds = total_birds.reset_index()
total_birds.columns=['observation_year', 'total_count_by_year']
# merging back
usfws_top10 = pd.merge(usfws_top10, total_birds, on='observation_year', how='inner')

# adding perc column
usfws_top10['perc_total_sightings'] = round(usfws_top10['count']/usfws_top10.total_count_by_year*100,2)

usfws_top10

,observation_year,common_name,count,total_count_by_year,perc_total_sightings
0,2007,American Crow,70,525,13.33
1,2007,Bald Eagle,16,525,3.05
2,2007,Canada Goose,29,525,5.52
3,2007,Carolina Wren,66,525,12.57
4,2007,Great Blue Heron,35,525,6.67
...,...,...,...,...,...
95,2016,Killdeer,395,5153,7.67
96,2016,Mallard,467,5153,9.06
97,2016,Northern Cardinal,587,5153,11.39
98,2016,Red-bellied Woodpecker,488,5153,9.47


In [96]:
# creating subset called usfws next top 10 (11-20), grouping by year & common name
usfws_next_top10 = birds_usfws.loc[birds_usfws.common_name.isin(['Turkey Vulture', 'Double-crested Cormorant', 'Blue Jay', 'Ring-billed Gull', 'Carolina Chickadee', 'Red-winged Blackbird', 'Eastern Bluebird', 'Pied-billed Grebe', 'Great Egret', 'Mourning Dove'])]
usfws_next_top10 = usfws_next_top10.groupby(['observation_year', 'common_name']).count()[['global_unique_identifier']].reset_index()
usfws_next_top10.columns = ['observation_year', 'common_name', 'count']

# adding total count per year as a column
total_birds = usfws_next_top10.groupby('observation_year').sum()[['count']]
total_birds = total_birds.reset_index()
total_birds.columns=['observation_year', 'total_count_by_year']
# merging back
usfws_next_top10 = pd.merge(usfws_next_top10, total_birds, on='observation_year', how='inner')

# adding perc column
usfws_next_top10['perc_total_sightings'] = round(usfws_next_top10['count']/usfws_next_top10.total_count_by_year*100,2)
usfws_next_top10

,observation_year,common_name,count,total_count_by_year,perc_total_sightings
0,2007,Blue Jay,37,253,14.62
1,2007,Carolina Chickadee,53,253,20.95
2,2007,Double-crested Cormorant,17,253,6.72
3,2007,Eastern Bluebird,23,253,9.09
4,2007,Great Egret,15,253,5.93
...,...,...,...,...,...
95,2016,Mourning Dove,365,4079,8.95
96,2016,Pied-billed Grebe,343,4079,8.41
97,2016,Red-winged Blackbird,406,4079,9.95
98,2016,Ring-billed Gull,418,4079,10.25


In [61]:
usfws_top10.groupby('common_name').sum()

,observation_year,count
common_name,,
American Crow,20115,2171
Bald Eagle,20115,1579
Canada Goose,20115,1807
Carolina Wren,20115,1753
Great Blue Heron,20115,2399
Killdeer,20115,1945
Mallard,20115,1750
Northern Cardinal,20115,2002
Red-bellied Woodpecker,20115,1603


In [ ]:
# count of sightings by observation year & common name
birds.groupby(['observation_year', 'common_name']).count()

### attempting webscraping:

come back to later

In [ ]:
# come back to later!!
# download Selenium webdriver

In [ ]:
# # importing beautiful soup
# import requests
# from bs4 import BeautifulSoup as BS

In [ ]:
# # establishing connection
# URL = 'https://www.fws.gov/refuge/tennessee/map'
# response = requests.get(URL)

In [ ]:
# # checking connection
# response.status_code

In [ ]:
# soup = BS(response.text)

In [ ]:
# soup.findAll('path', attrs={'class':'leaflet-interactive'})